# Detecting Fraud on Ethereum

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import data from data/transactions.csv
transactions_df = pd.read_csv('data/transactions.csv')

# show summary statistics
transactions_df.describe()

,nonce,transaction_index,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_status,block_number
count,500.000000,500.000000,5.000000e+02,5.000000e+02,5.000000e+02,5.000000e+02,500.000000,0.0,5.000000e+02
mean,14140.364000,3.740000,5.377770e+17,2.554518e+05,1.126978e+11,2.023732e+05,79296.202000,NaN,1.037778e+06
std,16914.389548,8.659004,5.120477e+18,1.900615e+05,7.689643e+11,1.865463e+05,49223.494149,NaN,1.388894e+03
min,0.000000,0.000000,0.000000e+00,2.200000e+04,4.545455e+10,2.161200e+04,21000.000000,NaN,1.035159e+06
25%,5248.750000,0.000000,1.000000e+00,2.500000e+05,5.200000e+10,8.125400e+04,80440.000000,NaN,1.036617e+06
50%,14984.500000,1.000000,1.000000e+00,2.500000e+05,5.200000e+10,1.617580e+05,80879.000000,NaN,1.037870e+06
75%,15109.250000,2.000000,1.000000e+00,2.500000e+05,5.200000e+10,2.426370e+05,81254.000000,NaN,1.038934e+06
max,198983.000000,54.000000,1.000000e+20,3.000000e+06,1.000000e+13,1.261261e+06,960234.000000,NaN,1.040213e+06


In [5]:
# create a copy of the df to explore and manipulate #
df = transactions_df.copy()

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   hash                         500 non-null    object 
 1   nonce                        500 non-null    int64  
 2   transaction_index            500 non-null    int64  
 3   from_address                 500 non-null    object 
 4   to_address                   495 non-null    object 
 5   value                        500 non-null    float64
 6   gas                          500 non-null    int64  
 7   gas_price                    500 non-null    int64  
 8   input                        500 non-null    object 
 9   receipt_cumulative_gas_used  500 non-null    int64  
 10  receipt_gas_used             500 non-null    int64  
 11  receipt_contract_address     5 non-null      object 
 12  receipt_root                 500 non-null    object 
 13  receipt_status      

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0xde4de80e40768af70f61ba4a2fd47cea10e147bd1de5...,1,1,0x1f5898f1eeab7a4c2015270ab3a1cd1ae37c651f,0x31eddd9732e2f47dd7da3a2d88d4c2b3e150e0ee,0.000000e+00,150789,45454545454,0xa9059cbb00000000000000000000000074afe54902d6...,71789,50789,NaN,0x4c78afa39a241f1bf790a5361e76a8b9ab12ec7b5327...,NaN,2/21/2016 20:45:11,1039542,0x2dd2a5adcddf14a63f1bd83d6a1556fce8afc61e690f...
1,0xdfadec9004313453c310860ccd2164b812c2a55167e2...,9,0,0xb3e3429824d31ba87edea73fac823a67f44c6015,0x7011f3edc7fa43c81440f9f43a6458174113b162,1.000000e+18,176660,80525500000,0x,76660,76660,NaN,0xd161292c649979ee70c8439ebbc6a3bf84db8c2f96ab...,NaN,2/21/2016 3:04:04,1035847,0x56038f7f446c2c1b39322bd0f61b33e8ce57cee93bcb...
2,0x745d50537bd1b13a5e849f35369b144cd1f114929e7a...,8,2,0xb3e3429824d31ba87edea73fac823a67f44c6015,0xfd2487cc0e5dce97f08be1bc8ef1dce8d5988b4d,1.000000e+18,192281,45454545454,0x,134281,92281,NaN,0x3664831e23c80a02cf4a21f7f2f960e0ee6fcc3c657a...,NaN,2/21/2016 2:24:34,1035690,0x8365b551e9fc1d311bd41bd71a654e0015a9fff197d9...
3,0x73a0b60096eaa4742b2f3b791206b6a13ffe5e3755cf...,0,0,0x1f5898f1eeab7a4c2015270ab3a1cd1ae37c651f,NaN,0.000000e+00,422428,45454545454,0x606060405260405161047e38038061047e8339810160...,322428,322428,0x31eddd9732e2f47dd7da3a2d88d4c2b3e150e0ee,0x7a599072039f949cac753a21c6b1b8e81d66c2f3c83d...,NaN,2/21/2016 20:36:16,1039511,0xe6cd9265f5b1c2b50ae66ea95a7d974b44920ccc5a30...
4,0x858ff8e9776ec0c351b64ed6792089eab53ff9f0a405...,66,39,0x8d10ebe7b7d812addc1a34bd38c9a2cf09ffb29c,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,1.945860e+18,22050,60717758102,0x,840000,21000,NaN,0xd1565012ef2e74858d24910d658f6c35ce36a34d9ea8...,NaN,2/21/2016 4:54:00,1036232,0x880a76d8caa8e2cf06606b355259ba818973c832c7d6...


In [9]:
# receipt_status column has no non null entries, this feature was added later in the creation of the ledger, 
# transactions with null entries for receipt_status were excecuted before the addition of this feature
# dropping this column is ok

null_status = df['receipt_status'].isna().value_counts()
print(f"Number of entries with null receipt_status: {null_status}")
df = df.drop("receipt_status", axis=1)

Number of entries with null receipt_status: True    500
Name: receipt_status, dtype: int64


In [24]:
# exploration of columns with null/missing values:

# to_address is missing 5/500 values, receipt_contract_address is missing 495/500
# exploration to see if this indicates directionality of the transaction

null_to_address = df[df['to_address'].isnull()].index.tolist()
null_receipt_address = df[df['receipt_contract_address'].isnull()].index.tolist()
print(f"Entries missing to_address: {null_to_address}")

print("Entries missing both to_address and receipt_contract_address:" + str([x for x in null_receipt_address if x in null_to_address]) + '\n')
for idx in null_to_address:
    receipt_address = df['receipt_contract_address'].iloc[idx]
    print(f"Receipt contract address for entry missing to_address: {receipt_address}")

print("\nNumber of missing receipt addresses: " + str(len(null_receipt_address)))
print("Entries missing a to_address have a receipt_contract_address, entries missing a receipt_contract_address have a to_address, these features are mutually exclusive.")

Entries missing to_address: [3, 19, 21, 24, 29]
Entries missing both to_address and receipt_contract_address:[]

Receipt contract address for entry missing to_address: 0x31eddd9732e2f47dd7da3a2d88d4c2b3e150e0ee
Receipt contract address for entry missing to_address: 0xc40f4163e53c714357bca9a8a28638b1c0b3c378
Receipt contract address for entry missing to_address: 0x0d10d2094abd0824d9bc9ef77a8c502a5e179ca1
Receipt contract address for entry missing to_address: 0x38acb2cb5c5f27f2d99cd89a5d410cb109c5a0ec
Receipt contract address for entry missing to_address: 0x19b858848e8b3813286ecaadc40586343ae98685

Number of missing receipt addresses: 495
Entries missing a to_address have a receipt_contract_address, entries missing a receipt_contract_address have a to_address, these features are mutually exclusive.


In [33]:
# Consider usable formats for features #

# Hash is a string used to identify transactions - can use index instead (?)

# what is nonce? what can this tell us about each transaction?
print(f"Max nonce: {df['nonce'].max()}")
print(f"Min nonce: {df['nonce'].min()}")
print(f"Mean nonce: {df['nonce'].mean()}")
print(f"Number of unique nonce values: {df['nonce'].nunique()}")

Max nonce: 198983
Min nonce: 0
Mean nonce: 14140.364
Number of unique nonce values: 465
